In [ ]:
!pip install opencv-contrib-python
!pip install --upgrade google-cloud-vision
!pip install utils

In [ ]:
import os
import io
import re
import math

import numpy as np
import pandas as pd

from PIL import ImageFont, ImageDraw, Image
import platform
import matplotlib as plt

from PIL import Image
import cv2
from google.cloud import vision

In [ ]:
#구글 api 불러오기 설정
def plt_imshow(title='image', img=None, figsize=(8 ,5)):
    plt.figure(figsize=figsize)

    if type(img) == list:
        if tyape(title) == list:
            titles = title
        else:
            titles = []

            for i in range(len(img)):
                titles.append(title)

        for i in range(len(img)):
            if len(img[i].shape) <= 2:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_GRAY2RGB)
            else:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)

            plt.subplot(1, len(img), i + 1), plt.imshow(rgbImg)
            plt.title(titles[i])
            plt.xticks([]), plt.yticks([])

        plt.show()
    else:
        if len(img.shape) < 3:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        plt.imshow(rgbImg)
        plt.title(title)
        plt.xticks([]), plt.yticks([])
        plt.show()

In [ ]:
#구글 api 불러오기 설정
# 폰트, 글꼴, 컬러
def putText(image, text, x, y, color=(0, 255, 0), font_size=22):
    if type(image) == np.ndarray:
        color_coverted = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(color_coverted)

    if platform.system() == 'Darwin':
        font = 'AppleGothic.ttf'
    elif platform.system() == 'Windows':
        font = 'malgun.ttf'
    else:
        font = 'NanumGothic.ttf'

    image_font = ImageFont.truetype(font, font_size)
    font = ImageFont.load_default()
    draw = ImageDraw.Draw(image)

    draw.text((x, y), text, font=image_font, fill=color)

    numpy_image = np.array(image)
    opencv_image = cv2.cvtColor(numpy_image, cv2.COLOR_RGB2BGR)

    return opencv_image

In [ ]:
# 임의의 사각형을 이루는 각 변의 벡터 계산
# title_rate 계산을 위해 필요

def calculate_polygon_area(vertices):
    vectors = []

    for i in range(len(vertices)):
        x1, y1 = vertices[i]
        x2, y2 = vertices[(i + 1) % len(vertices)]
        vectors.append((x2 - x1, y2 - y1))

    # 외적을 통한 넓이 계산
    area = 0
    for i in range(len(vectors)):
        area += vectors[i][0] * vectors[(i + 1) % len(vectors)][1]
        area -= vectors[i][1] * vectors[(i + 1) % len(vectors)][0]

    area = abs(area) / 2
    return area

In [ ]:
# 이미지 크기 계산
def size(img):
    im = Image.open('my_path'+img) # 여기
    width, height = im.size

    return width*height

In [ ]:
# 구글 클라우드 연동!
# 키 값 입력

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "secret_key" # 키

client_options = {'api_endpoint': 'eu-vision.googleapis.com'}
client = vision.ImageAnnotatorClient(client_options=client_options)

In [ ]:
def make_list(directory,thumb_idx,image_index):
    # thumb_idx :  썸네일 이미지의 순서
    lst=[]

    for image_name, index in image_index.items():
        print("<",index,image_name,">")
        path=directory+"/"+image_name

        with io.open(path, 'rb') as image_file:
            content = image_file.read()

        client = vision.ImageAnnotatorClient()

        # 이미지 컨텐츠 설정
        image = vision.Image(content=content)

        # 이미지 컨텍스트 설정 (언어 힌트 추가)
        image_context = vision.ImageContext(language_hints=["ko"])

        # 텍스트 감지 요청
        response = client.text_detection(image=image, image_context=image_context)
        texts = response.text_annotations

            # texts에서 글씨 이상한 경우 제거
            # texts = remove_special_characters(texts[0].description)

        poster_size = size(image_name)


        text_rate1=0 #글자별 비율
        for idx, text in enumerate(texts):
            vertices = [(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices]
            box_size = calculate_polygon_area(vertices)

            temp = []

            temp.append(thumb_idx) #썸네일 이미지의 인덱스
            temp.append(os.path.splitext(image_name)[0]+'.png') # 썸네일 이미지 제목
            temp.append(poster_size) # 썸네일 이미지 전체 사이즈
            temp.append(idx)
            temp.append(text.description) #인식한 단어
            temp.append(len(text.description)) #인식한 단어 전체 길이(한 음절 기준)
            temp.append(vertices[0]) #좌표 좌측 하단
            temp.append(vertices[1]) #좌표 우측 하단
            temp.append(vertices[2]) #좌표 우측 상단
            temp.append(vertices[3]) #좌표 좌측 상단
            temp.append(calculate_polygon_area(vertices)) #단어 상자 크기_ 글자 크기로 간주

            if idx!=0:
                text_rate1+=box_size/poster_size
            temp.append(text_rate1) # 전체 이미지 크기 대비 단어 상자별 비율



        lst.append(temp)
        thumb_idx +=1

    return lst

In [ ]:
# 경로 설정 후 썸네일 이미지 리스트 받기
directory = "my_path"
files = sorted(os.listdir(directory))
image_index = {}

# # 파일 목록을 순회하며 이미지 파일인지 확인하고 인덱스 부여
for index, file in enumerate(files):
    file_extension = os.path.splitext(file)[1].lower()
    if file_extension in ['.png']:
        image_index[file] = index

In [ ]:
# thumb_idx 시작점 : 1
lst= make_list(directory,1,image_index)

In [ ]:
data = pd.DataFrame(lst,
                   columns = ['thumb_idx','videoId','total_size','idx','text','text_length',
                              'vertices[0]','vertices[1]','vertices[2]','vertices[3]','calculate_polygon_area(vertices)','text_rate1'])

In [ ]:
#csv 저장경로
data.to_csv('my_path', header=True, sep=',',encoding="utf-8") #여기

In [ ]:
#결과 확인
data

In [ ]:
data[data["text_length"].isnull()]

In [ ]:
# 글씨가 없는 경우 text가 null로 들어옴
# 결측치는 0으로 처리
data = data.fillna(0)